In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from skfuzzy.cluster import cmeans, cmeans_predict
from numpy import zeros, ones, empty, argmax, log, exp
from numpy.random import rand

In [2]:
def t(a, b):
    return a * b

In [3]:
def s(a, b):
    return a + b - a * b

In [4]:
def t_sc(a, b):
    return int(a) & int(b)

In [5]:
def s_sc(a, b):
    return int(a) | int(b)

In [6]:
dataset = load_iris()
tar = dataset.target_names.shape[0]
c = tar

In [7]:
m = 2
error = 0.001
maxiter = 1000
ha = 10
alpha = 1
epoch = 100
repeat = 10
l_sc = 1

In [8]:
for h in range(ha, 0, -1):
    avg_acc = 0
    for r in range(repeat):
        x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target)
        cntr, u, u0, d, jm, p, fpc = cmeans(x_train.T, c, m, error, maxiter)
        v = rand(c, h)
        w = rand(h, tar)
        N_train = x_train.shape[0]
        x = empty((N_train, c))
        for i in range(N_train):
            for j in range(c):
                x[i][j] = u[j][i]
        z = ones((N_train, h))
        y = zeros((N_train, tar))
        target_train = zeros((N_train, tar))
        for i in range(N_train):
            target_train[i][y_train[i]] = 1
        for e in range(epoch):
            for i in range(N_train):
                for j in range(h):
                    for k in range(c):
                        z[i][j] = t(z[i][j], s(x[i][k], v[k][j]))
                for j in range(tar):
                    for k in range(h):
                        y[i][j] = s(y[i][j], t(z[i][k], w[k][j]))
            for i in range(N_train):
                qwe = sum(exp(y[i]))
                for j in range(tar):
                    y[i][j] = exp(y[i][j]) / qwe
            dw = zeros((h, tar))
            for i in range(h):
                for j in range(tar):
                    for k in range(N_train):
                        A = 0
                        for l in range(h):
                            if l != i:
                                A = s(A, t(z[k][l], w[l][j]))
                        dw[i][j] += (y[k][j] - target_train[k][j]) * (z[k][i] * (1 - A))
            dv = zeros((c, h))
            for i in range(c):
                for j in range(h):
                    for k in range(N_train):
                        for l in range(tar):
                            A = 0
                            for m in range(h):
                                if m != j:
                                    A = s(A, t(z[k][m], w[m][l]))
                            B = 1
                            for m in range(c):
                                if m != i:
                                    B = t(B, s(x[k][m], v[m][j]))
                            dv[i][j] += (y[k][l] - target_train[k][l]) * (w[j][l] * (1 - A)) * (B * (1 - x[k][i]))
            w -= alpha * dw
            for i in range(len(w)):
                for j in range(len(w[i])):
                    if w[i][j] < 0:
                        w[i][j] = 0
                    elif w[i][j] > 1:
                        w[i][j] = 1
            v -= alpha * dv
            for i in range(len(v)):
                for j in range(len(v[i])):
                    if v[i][j] < 0:
                        v[i][j] = 0
                    elif v[i][j] > 1:
                        v[i][j] = 1
        u, u0, d, jm, p, fpc = cmeans_predict(x_test.T, cntr, m, error, maxiter)
        N_test = x_test.shape[0]
        x = empty((N_test, c))
        for i in range(N_test):
            for j in range(c):
                x[i][j] = u[j][i]
        z = ones((N_test, h))
        y = zeros((N_test, tar))
        target_test = zeros((N_test, tar))
        for i in range(N_test):
            target_test[i][y_test[i]] = 1
        v_sc = empty((c, h))
        for i in range(c):
            for j in range(h):
                seq = 0
                for b in range(l_sc):
                    seq <<= 1
                    seq += (v[i][j] > rand())
                v_sc[i][j] = seq
        w_sc = empty((h, tar))
        for i in range(h):
            for j in range(tar):
                seq = 0
                for b in range(l_sc):
                    seq <<= 1
                    seq += (w[i][j] > rand())
                w_sc[i][j] = seq        
        x_sc = empty((N_test, c))
        for i in range(N_test):
            for j in range(c):
                seq = 0
                for b in range(l_sc):
                    seq <<= 1
                    seq += (x[i][j] > rand())
                x_sc[i][j] = seq
        z_sc = empty((N_test, h))
        for i in range(N_test):
            for j in range(h):
                seq = 0
                for b in range(l_sc):
                    seq <<= 1
                    seq += True
                z_sc[i][j] = seq
        y_sc = empty((N_test, tar))
        for i in range(N_test):
            for j in range(tar):
                seq = 0
                for b in range(l_sc):
                    seq <<= 1
                    seq += False
                y_sc[i][j] = seq
        for i in range(N_test):
            for j in range(h):
                for k in range(c):
                    z_sc[i][j] = t_sc(z_sc[i][j], s_sc(x_sc[i][k], v_sc[k][j]))
            for j in range(tar):
                for k in range(h):
                    y_sc[i][j] = s_sc(y_sc[i][j], t_sc(z_sc[i][k], w_sc[k][j]))
        sc = empty((N_test, tar))
        for i in range(N_test):
            for j in range(tar):
                n = 0
                for b in range(l_sc):
                    n += (int(y_sc[i][j]) >> b) % 2
                sc[i][j] = n / l_sc
        target_sc = zeros((N_test, tar))
        for i in range(N_test):
            target_sc[i][argmax(sc[i])] = 1
        count = 0
        for i in range(N_test):
            for j in range(tar):
                if target_test[i][j] == target_sc[i][j]:
                    count += 1
        count /= tar
        avg_acc += count / N_test
    avg_acc /= repeat
    avg_acc = round(avg_acc * 100)
    print(avg_acc)

78
79
76
80
81
81
75
77
69
63
